In [44]:
import sqlite3 as sql
import altair as alt
import json
import plotly.express as px
from collections import Counter
from IPython.display import SVG
import csv
import pprint

In [45]:
db_file = '/Users/maxime/Documents/dossier_python/generic_database.sqlite'

In [46]:
cn = sql.connect(db_file)
# Le contenu de la variable: connexion
cn

In [47]:
q2 = """
SELECT SUBSTR(text_value, 1,4)  AS annee, COUNT(*) AS effectif, 
GROUP_CONCAT(SUBSTRING(subject_uri, LENGTH('http://dbpedia.org/resource/') + 1, LENGTH(subject_uri)), ', ')
FROM "statement" s 
WHERE subject_uri LIKE '%dbpe%'
GROUP BY SUBSTR(text_value, 1,4) ;
"""

In [48]:
cur = cn.cursor()
cur

In [49]:
cur.execute(q2)

r = cur.fetchall()

In [50]:
print(f'Effectif: {len(r)}, type de valeur: {type(r)}')
r[:3]

Effectif: 351, type de valeur: <class 'list'>


[('0940', 1, "Abu_al-Wafa'_Buzjani"),
 ('1048', 1, 'Omar_Khayyam'),
 ('1256', 1, "Ibn_al-Banna'_al-Marrakushi")]

In [51]:
la = []
for e in r[1:]:
    la.append(int(e[0][:4]))
la[:3]

[1048, 1256, 1271]

In [52]:
la = []
for e in r:
    la.append(int(e[0][:4]))
la[:3]

[940, 1048, 1256]

In [53]:
ys = [y[0] for y in r]; print (ys[:3], ys[-3:])
min(ys), max(ys)

['0940', '1048', '1256'] ['1989', '1991', '1993']


('0940', '1993')

In [54]:
la = []
for e in r:
    la.append(int(e[0][:4]))
la[:3]

[940, 1048, 1256]

In [55]:
ctr = Counter(la)
list(ctr.items())[:7]

[(940, 1), (1048, 1), (1256, 1), (1271, 1), (1332, 1), (1423, 1), (1436, 1)]

In [56]:
### Longueur, type du résultat et premières lignes du résultat
print(f'Effectif: {len(r)}, type de valeur: {type(r)}')
r[:3]

Effectif: 351, type de valeur: <class 'list'>


[('0940', 1, "Abu_al-Wafa'_Buzjani"),
 ('1048', 1, 'Omar_Khayyam'),
 ('1256', 1, "Ibn_al-Banna'_al-Marrakushi")]

In [57]:
### Longueur, type du résultat et premières lignes du résultat
print(f'Effectif: {len(r)}, type de valeur: {type(r)}')
la[:3]

Effectif: 351, type de valeur: <class 'list'>


[940, 1048, 1256]

In [58]:
r[:3]

[('0940', 1, "Abu_al-Wafa'_Buzjani"),
 ('1048', 1, 'Omar_Khayyam'),
 ('1256', 1, "Ibn_al-Banna'_al-Marrakushi")]

In [59]:
merged_list = [(la[i], r[i]) for i in range(0, len(r))]
merged_list[:3]

[(940, ('0940', 1, "Abu_al-Wafa'_Buzjani")),
 (1048, ('1048', 1, 'Omar_Khayyam')),
 (1256, ('1256', 1, "Ibn_al-Banna'_al-Marrakushi"))]

In [60]:
min(la), max(la)

(940, 1993)

In [61]:
y_l = list(range(min(la), max(la), 1)); len(y_l),y_l[:4],y_l[-4:]

(1053, [940, 941, 942, 943], [1989, 1990, 1991, 1992])

In [89]:
i=0
r_int  = []
for b in y_l:
    l = []

    for v in la:
        if b == v:
            l = [b] + list(r[i][1:3])
            i=i+1
            
            r_int.append(l)
    
r_int[:10]

[[940, 1, "Abu_al-Wafa'_Buzjani"],
 [1048, 1, 'Omar_Khayyam'],
 [1256, 1, "Ibn_al-Banna'_al-Marrakushi"],
 [1271, 1, 'Zhao_Youqin'],
 [1332, 1, 'Ibn_Khaldun'],
 [1423, 2, 'Georg_von_Peuerbach, Antonio_Manetti'],
 [1436, 1, 'Regiomontanus'],
 [1444, 1, 'Nilakantha_Somayaji'],
 [1452, 1, 'Abraham_Zacuto'],
 [1465, 1, 'Scipione_del_Ferro']]

In [90]:
i=0
y_r  = []
for a in y_l:
    l = []

    for v in la:
        if a == v:
            l = [a] + list(r[i][1:3])
            i=i+1
            pass

    if len(l) > 2:
        y_r.append(l)
    else:
        y_r.append([a,0,''])
y_r[:10]

[[940, 1, "Abu_al-Wafa'_Buzjani"],
 [941, 0, ''],
 [942, 0, ''],
 [943, 0, ''],
 [944, 0, ''],
 [945, 0, ''],
 [946, 0, ''],
 [947, 0, ''],
 [948, 0, ''],
 [949, 0, '']]

In [77]:
data = [{'year': e[0], 'eff': e[1], 'names': e[2]} for e in y_r]

In [78]:
alt.Chart.from_dict({
    "data": {
    "values" : data    
    # "url" : "http://localhost:8888/files/astronomers/notebooks_jupyter/dbpedia_exploration/data/birth_years_grouped_y.json"
    },
    "mark": "bar",
    # {
    #     "type":"bar",
    #     "tooltip": {"content": "data"}
    # }, 
    "encoding": {
        "x": {"field": "year", "type": "ordinal", 
              "axis": {"values": [900,1000, 1100, 1200, 1300, 1400, 1500, 1600, 1700 , 1800, 1900, 2000]}}, # ordinal, quantitative
         "y": {"field": "eff", "type": "quantitative"},
        "tooltip": [
      {"field": "year", "type": "quantitative", "title": "Year"},
     {"field": "names", "type": "nominal", "title": "Names"}
    ]
    },
    "width": 1000, 
    "height": 300
}

)

alt.Chart(...)

In [66]:
ys = [y[0] for y in r]; print (ys[:3], ys[-3:])
min(ys), max(ys)

['0940', '1048', '1256'] ['1989', '1991', '1993']


('0940', '1993')

In [92]:
periode = 20
per_l = list(range(940, 1993, periode))
per_ll = [(l, l+ periode -1) for l in per_l]
len(per_ll),per_ll[:3],per_ll[-3:]

(53,
 [(940, 959), (960, 979), (980, 999)],
 [(1940, 1959), (1960, 1979), (1980, 1999)])

In [115]:
per_r = []
for a in per_ll:
    #print(a)
    if a[1] < 1000:
        label = f'0{a[0]}_0{a[1]}'
    else:
        label = f'{a[0]}_{a[1]}'
    eff = 0 
    l = []
    for v in r_int:
        #print(v)
        if v[0] >= a[0] and v[0] <= a[1] :
            eff = eff + v[1]
            l.extend(v[2].split(','))               

    per_r.append([label, eff, l])

per_r[:10]        

[['0940_0959', 1, ["Abu_al-Wafa'_Buzjani"]],
 ['0960_0979', 0, []],
 ['0980_0999', 0, []],
 ['1000_1019', 0, []],
 ['1020_1039', 0, []],
 ['1040_1059', 1, ['Omar_Khayyam']],
 ['1060_1079', 0, []],
 ['1080_1099', 0, []],
 ['1100_1119', 0, []],
 ['1120_1139', 0, []]]

In [116]:
labels = ['<br>'.join([m.strip() for m in i[2]]) for i in per_r] ; len(labels),labels[0]

(53, "Abu_al-Wafa'_Buzjani")

In [117]:
labels = [[m.strip() for m in i[2]] for i in per_r] ; len(labels),labels[0]

(53, ["Abu_al-Wafa'_Buzjani"])

In [118]:
data = [{'per': e[0], 'eff': e[1], 'names': '\n'.join(e[2])} for e in per_r]

In [119]:


### Ajouter une moyenne glissante
# Documentation:
#     https://joelostblom.github.io/altair-docs/gallery/bar_with_rolling_mean.html

## https://altair-viz.github.io/user_guide/data.html
data_trsf = alt.Data(values=data)

bar = alt.Chart(data_trsf).mark_bar().encode(
    x='per:N',
    y='eff:Q',
    tooltip=('eff:Q', 'names:N')
   
)

line = alt.Chart(data_trsf).mark_line(color='red').transform_window(
    # The field to average
    rolling_mean='mean(eff)',
    # The number of values before and after the current value to include.
    frame=[-9, 0]
).encode(
    x='per:O',
    y='rolling_mean:Q'
)

(bar + line).properties(height= 600, width=1000,  title = 'Distribution des naissances par périodes de vingt ans')



alt.LayerChart(...)